In [ ]:
COMP_NAME = 'predict-west-nile-virus'

In [ ]:
from fastai.tabular.all import *

In [ ]:
WORKING_DIR = Path(".").absolute()
INPUT_DIR = Path("../input").absolute()
ARCHIVE_DIR = Path("../archive").absolute()
MODELS_DIR = Path("../models").absolute()

In [ ]:
DATA_DIR = Path("../data").absolute()
submission_path = DATA_DIR

In [ ]:
path = Path(INPUT_DIR/COMP_NAME); Path.BASE_PATH = path; path.ls()

In [ ]:
import pandas as pd
import numpy as np
from sklearn import ensemble, preprocessing

In [ ]:
# Load dataset 
# train = pd.read_csv('../input/train.csv')
# test = pd.read_csv('../input/test.csv')
# sample = pd.read_csv('../input/sampleSubmission.csv')
# weather = pd.read_csv('../input/weather.csv')

In [ ]:
train = pd.read_csv(path/'train.csv')
test = pd.read_csv(path/'test.csv')
sample = pd.read_csv(path/'sampleSubmission.csv')
weather = pd.read_csv(path/'weather.csv')

In [ ]:
# Get labels
labels = train.WnvPresent.values

In [ ]:
# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

In [ ]:
# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [ ]:
# replace some missing values and T with -1
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)

In [ ]:
# Functions to extract month and day from dataset
# You can also use parse_dates of Pandas.
def create_month(x):
    return x.split('-')[1]

def create_day(x):
    return x.split('-')[2]

In [ ]:
train['month'] = train.Date.apply(create_month)
train['day'] = train.Date.apply(create_day)
test['month'] = test.Date.apply(create_month)
test['day'] = test.Date.apply(create_day)

In [ ]:
# Add integer latitude/longitude columns
train['Lat_int'] = train.Latitude.apply(int)
train['Long_int'] = train.Longitude.apply(int)
test['Lat_int'] = test.Latitude.apply(int)
test['Long_int'] = test.Longitude.apply(int)

In [ ]:
# drop address columns
train = train.drop(['Address', 'AddressNumberAndStreet','WnvPresent', 'NumMosquitos'], axis = 1)
test = test.drop(['Id', 'Address', 'AddressNumberAndStreet'], axis = 1)

In [ ]:
# Merge with weather data
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

In [ ]:
# Convert categorical data to numbers
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train['Species'].values) + list(test['Species'].values))
train['Species'] = lbl.transform(train['Species'].values)
test['Species'] = lbl.transform(test['Species'].values)

In [ ]:
lbl.fit(list(train['Street'].values) + list(test['Street'].values))
train['Street'] = lbl.transform(train['Street'].values)
test['Street'] = lbl.transform(test['Street'].values)

In [ ]:
lbl.fit(list(train['Trap'].values) + list(test['Trap'].values))
train['Trap'] = lbl.transform(train['Trap'].values)
test['Trap'] = lbl.transform(test['Trap'].values)

In [ ]:
# drop columns with -1s
train = train.loc[:,(train != -1).any(axis=0)]
test = test.loc[:,(test != -1).any(axis=0)]

In [ ]:
# Random Forest Classifier 
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=2)
clf.fit(train, labels)

In [ ]:
# create predictions and submission file
predictions = clf.predict_proba(test)[:,1]
sample['WnvPresent'] = predictions
!mkdir -p {submission_path}/abishek-beat-the-benchmark
sample.to_csv(submission_path/'abishek-beat-the-benchmark'/'submission1.csv', index=False)

In [ ]:
sample.head()